In [16]:
%matplotlib inline
import os
import glob
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pathlib
from matplotlib.lines import Line2D
from PIL import Image
import arff


In [17]:
_METRICS_FILE = './files/benchmark_sensitivityAnalysis2'
_METRIC = 'timeperinstance'
_NUM_CLASSIFIER =  10

directory = './files/statistic_test/'+_METRIC+'/'


datasets_to_remove = ['bankmarketingnorm', 'airlinesnorm']
datasets_to_remove += ['breast', 'carnorm', 'covtype', 'electricity', 'higgs200k', 'kddcup99norm', 'ozone', 'pendigitis', 'pokerhand', 'spambase', 'susy100k']
classifiers_to_remove = ['keras_3_Dilated_Conv', 'keras_parallel_3_Dilated_Conv', 'MOA_SGD', 'MOA_WeightedMajorityAlgorithm']

In [18]:
classifier_labels_full = {
    'keras_parallel_keras_parallel_3_Dilated_Conv_pooling_60x90':'ADLStream',
    'sklearn_MLPClassifier':'MLP',
    'MOA_NaiveBayes':'NB',
    'MOA_SingleClassifierDrift':'SCD',
    'MOA_ASHoeffdingTree':'ASHT',
    'MOA_HoeffdingTree':'HT',
    'MOA_HoeffdingOptionTree':'HT',
    'MOA_LeveragingBag':'LBAG',
    'MOA_RandomHoeffdingTree':'RHT',
    'MOA_OzaBoostAdwin':'BO-AD',
    'MOA_OzaBoost':'BO',
    'sklearn_BernoulliNB':'BNB',
    'MOA_HoeffdingAdaptiveTree':'HAT',
    'MOA_ActiveClassifier':'AC',
    'MOA_OzaBag':'BA',
    'MOA_AdaHoeffdingOptionTree':'ADHT',
    'MOA_OzaBagAdwin':'BA-AD',
    'MOA_OzaBagASHT':'B-ASHT',
    'sklearn_PassiveAggressiveClassifier':'PAC',
    'sklearn_SGDClassifier':'SGD',
    'sklearn_Perceptron':'P',
    'MOA_DecisionStump':'DST',
    'MOA_OCBoost':'OCBO',
    'MOA_SPegasos':'SPEG',
    'MOA_MajorityClass':'MC',
    'MOA_Perceptron':'P',
    'MOA_KUE': 'KUE',
    'MOA_AdaptiveRandomForest':'ARF'
}

datasets = ['TwoPatterns', 'CinCECGtorso', 'TwoLeadECG', 'Wafer', 'pendigits', 'FacesUCR', 'Mallat', 'FaceAll', 'Symbols', 'ItalyPowerDemand', 'ECG5000', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 'NonInvasiveFetalECGThorax2', 'SwedishLeaf', 'FordA', 'Yoga', 'UWaveGestureLibraryX', 'FordB', 'ElectricDevices', 'UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'HandOutlines', 'InsectWingbeatSound', 'ShapesAll', 'MedicalImages', 'PhalangesOutlinesCorrect', 'ChlorineConcentration', 'Phoneme']

In [19]:
# read data
metrics = pd.read_csv(_METRICS_FILE+'.csv')

# Delete datasets and classifier that we dont want
metrics = metrics[~metrics.classifier.isin(classifiers_to_remove)]
metrics = metrics[~metrics.dataset.isin(datasets_to_remove)]
metrics = metrics[metrics['instances']>1000]

metrics = metrics[metrics.classifier.isin(['MOA_ASHoeffdingTree', 'MOA_AdaptiveRandomForest', 'MOA_OzaBoostAdwin', 'MOA_SingleClassifierDrift', 'keras_parallel_keras_parallel_3_Dilated_Conv_pooling_60x90', 'sklearn_MLPClassifier', 'sklearn_SGDClassifier', 'MOA_NaiveBayes', 'MOA_LeveragingBag'])]

print(len(metrics.dataset.unique()), "datasets")
metrics.sample(10)

29 datasets


,dataset,classifier,instances,attributes,classes,accuracy,kappa,train_time_mean,test_time_mean,total_time_mean,train_time,test_time,total_time
7635,Symbols,MOA_SingleClassifierDrift,1020,398,6,0.913063,0.889887,0.001271,0.001189,0.002459,0.129595,0.121234,0.250829
4032,ItalyPowerDemand,MOA_AdaptiveRandomForest,1096,24,2,0.927731,0.841356,0.000909,0.000039,0.000948,0.099965,0.004296,0.104261
1471,ECG5000,MOA_NaiveBayes,5000,140,5,0.863463,0.750474,0.000026,0.000349,0.000374,0.012776,0.174279,0.187055
2243,FacesUCR,MOA_ASHoeffdingTree,2250,131,14,0.807631,0.777421,0.001146,0.000910,0.002055,0.257816,0.204646,0.462462
4647,Mallat,MOA_SingleClassifierDrift,2400,1024,8,0.944707,0.931191,0.004779,0.004545,0.009324,1.146913,1.090817,2.237730
8703,Wafer,MOA_LeveragingBag,7164,152,2,0.993831,0.963474,0.007037,0.000501,0.007537,5.045334,0.359030,5.404364
6471,Phoneme,keras_parallel_keras_parallel_3_Dilated_Conv_p...,2020,1024,39,0.249366,0.189648,0.159927,0.112465,0.272392,3.678332,2.586684,3.678332
2838,FordB,MOA_ASHoeffdingTree,4446,500,2,0.521095,0.067062,0.001093,0.000537,0.001630,0.486228,0.239030,0.725258
2260,FacesUCR,MOA_OzaBoostAdwin,2250,131,14,0.891921,0.873506,0.019330,0.008868,0.028197,4.349152,1.995202,6.344354
5789,NonInvasiveFetalECGThorax1,keras_parallel_keras_parallel_3_Dilated_Conv_p...,3675,750,42,0.904013,0.893001,0.169387,0.096469,0.265856,6.944861,3.955230,6.944861


In [20]:
metrics_table_kappa = metrics.pivot(index='dataset',columns='classifier')['kappa']
metrics_table_kappa = metrics_table_kappa.dropna()
metrics_table_kappa = metrics_table_kappa.reindex(metrics_table_kappa.mean().sort_values(ascending=False).index, axis=1)
metrics_table_kappa = metrics_table_kappa.sort_values(by=list(metrics_table_kappa.columns), ascending=False)

metrics['instancespersecond'] = metrics['instances']/metrics['total_time']
metrics['timeperinstance'] = metrics['total_time']/metrics['instances']*1000
metrics['minutes'] = metrics['total_time']/60

metrics_table = metrics.pivot(index='dataset',columns='classifier')[_METRIC]
# metrics_table = metrics_table.dropna()
metrics_table = metrics_table.reindex(metrics_table_kappa.mean().sort_values(ascending=False).index, axis=1)
#metrics_table = metrics_table.sort_values(by=list(metrics_table_kappa.columns), ascending=False)
metrics_table = metrics_table.reindex(index=metrics_table_kappa.index)
metrics_table = metrics_table[metrics_table.columns[0:]]
metrics_table.columns = [classifier_labels_full[clf] for clf in metrics_table.columns]
metrics_table = metrics_table.loc[datasets]

print(metrics_table.shape)
metrics_table


(29, 9)


,ADLStream,ARF,MLP,NB,SCD,ASHT,LBAG,BO-AD,SGD
dataset,,,,,,,,,
TwoPatterns,1.587400,0.257645,0.094113,0.028720,0.049070,0.064636,1.308990,0.520365,0.056452
CinCECGtorso,2.296192,1.286605,0.866581,0.380395,0.699634,0.851023,15.543146,7.299272,0.241942
TwoLeadECG,1.902573,0.154688,0.079045,0.010149,0.016725,0.023391,0.468204,0.362255,0.023638
Wafer,1.458948,0.086500,0.108857,0.051860,0.035166,0.034374,0.754378,0.590655,0.032772
pendigits,1.241148,0.325851,0.070988,0.008607,0.017107,0.016974,0.331867,0.236476,0.079704
FacesUCR,1.748797,0.816974,0.096534,0.091877,0.175441,0.205539,3.334478,2.819713,0.112125
Mallat,1.937608,1.073455,0.628570,0.481864,0.932388,1.068472,19.526912,11.152084,0.234667
FaceAll,1.640330,0.686577,0.094806,0.097782,0.182200,0.212948,3.370354,2.815929,0.112897
Symbols,1.940033,0.654345,0.402064,0.128869,0.245910,0.315360,6.622575,3.277581,0.170418


In [21]:
file_path = directory+_METRIC+'_table2.csv'
if not os.path.exists(directory):
    os.makedirs(directory)
metrics_table.to_csv(file_path, sep=',', encoding='utf-8')

In [22]:
with open(file_path, "r+") as f:
    lines = f.readlines()
    #del lines[2]
    #del lines[0]
    lines[0] = lines[0].replace('dataset',' ', 1 )
    f.seek(0)
    f.truncate()
    f.writelines(lines)

In [8]:
[c for c in metrics_table.columns]

['ADLStream', 'ARF', 'MLP', 'NB', 'SCD', 'ASHT', 'LBAG', 'BO-AD', 'SGD']